In [3]:
from datasets import load_dataset

math_dataset = load_dataset("hendrycks/competition_math")
math_dataset_train = math_dataset["train"]
math_dataset_test = math_dataset["test"]

In [55]:
import dspy
from dspy import Example
from openai import OpenAI
from dspy.teleprompt import BootstrapFewShot
import re
import os

client = OpenAI()

def find_closing_boxed(text):
    matches = re.finditer(r'\\boxed\{', text) # Find all answers
    start_index = [match.end() for match in matches][-1] # Get the last \boxed{ answer
    substring = text[start_index:]
    brace_count = 1

    for end_index, char in enumerate(substring, start = start_index):
        if char == '{':
            brace_count += 1
        elif char == '}':
            brace_count -= 1
        else:
            continue

        if brace_count == 0:
            return [start_index, end_index]
        
    # Return None if no matching brace is found (though this should not happen in well-formed strings)
    return [start_index, -1]

def extract_final_answer(math_solution):
    start_index, end_index = find_closing_boxed(math_solution)
    return math_solution[start_index:end_index].strip()

def simple_accuracy(model_solution, official_solution):
    return model_solution == official_solution




# model_final_answer = extract_final_answer(model_full_solution)
# offical_final_answer = extract_final_answer(rand_entry["solution"])


math_dataset_train = math_dataset["train"]
first_ten_train_dict = math_dataset_train.select(range(10))
first_ten_train_set = [Example(dict(item) for item in first_ten_train_dict).with_inputs("problem")]



def gsm8k_metric(gold, pred, trace=None) -> int:
    return int(parse_integer_answer(str(gold.answer))) == int(parse_integer_answer(str(pred.answer)))



turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=500)
dspy.settings.configure(lm=turbo)

cot_prompt = dspy.ChainOfThought("problem -> solution")


# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=simple_accuracy, **config)

optimized_cot = teleprompter.compile(cot_prompt, trainset=first_ten_train_set)

  0%|          | 0/1 [00:00<?, ?it/s]

ERROR:dspy.teleprompt.bootstrap:2024-07-09T01:12:22.673642Z [error    ] Failed to run or to evaluate example Example({}) (input_keys={'problem'}) with <function simple_accuracy at 0x1407f82c0> due to simple_accuracy() takes 2 positional arguments but 3 were given. [dspy.teleprompt.bootstrap] filename=bootstrap.py lineno=211
100%|██████████| 1/1 [00:02<00:00,  2.33s/it]

Bootstrapped 0 full traces after 1 examples in round 0.


In [43]:
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

100%|██████████| 1319/1319 [00:00<00:00, 73872.17it/s]


In [54]:
gsm8k_trainset

[Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'}),
 Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>6

In [52]:
first_ten_train_dict = math_dataset_train.select(range(10))
first_ten_train_set = [dict(item) for item in first_ten_train_dict].with_inputs("problem")

[{'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).',
  'level': 'Level 5',
  'type': 'Algebra',
  'solution': 'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'},
 {'problem': 'A rectangular band formation is a formation with $m$ band members in each of $r$ rows, where $m$ and $r$ are integers. A particular band has less than 100 band members. The director arranges them in a rectangular formation and finds that he has two 